In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
import dmpc


from eom import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
# import intelligent_robotics as ir
import numpy as np
from scipy.integrate import odeint
from sympy import simplify
from sympy import sin, cos, symbols, Matrix, solve
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
init_vprinting()

## System dynamics ##

The system to be controlled is an dual-moiton inverted pendulum on a cart (see next Figure).  

<img src="dual_motion.png" width="800" align="center"/>

Parameter description  
$m_t$ = 몸체 무게  
$m_c$ = 슬라이더 무게  
$m_w$ = 휠 무게  
$r$ = 휠 반지름  
$\theta_p $ = 몸체의 기울기(그림에서 $\theta$)  
$\theta_w $ = 휠 각도  
$x_l$ = 슬라이더 위치  
$h_t$ = 휠 중심부터 몸체 중심 까지의 거리  
$h_c$ = 휠 중심부터 슬라이더 중심 까지의 거리  
$I_t$ = 몸체 관성 모멘트  
$I_c$ = 슬라이더 관성 모멘트  
$I_w$ = 휠 관성 모멘트  

Lagrangian dynamics system parameter = $[\theta_w\; \theta_p\; x_l]$



In [2]:
r, h_t, h_c, t = sp.symbols('r, h_t, h_c, t')
m_w, m_wr, m_wl, m_t, m_c, g = sp.symbols('m_w, m_wr, m_wl, m_t, m_c, g')
I_w, I_t, I_c = sp.symbols('I_w, I_t, I_c')
x, theta_wl, theta_wr, theta_p, x_l = dynamicsymbols('x, theta_wl, theta_wr, theta_p, x_l')

In [3]:
# Displacement Vector
# x_wl = trans(r*theta_wl,0,0)
# x_wr = trans(r*theta_wr,0,0)
# x_c = (x_wr + x_wl)/2 + rot_y(theta_p) @ trans(0,0,h_c)
# x_t = (x_wr + x_wl)/2 + rot_y(theta_p) @ trans(x_l,0,h_t)
x_b = trans(x,0,0)
x_c = x_b + rot_y(theta_p) @ trans(0,0,h_c)
x_t = x_b + rot_y(theta_p) @ trans(x_l,0,h_t)

# Velocity Vector
# v_wl = sp.simplify(time_derivative(x_wl,N))
# v_wr = sp.simplify(time_derivative(x_wr,N))
v_c = sp.simplify(time_derivative(x_c,N))
v_t = sp.simplify(time_derivative(x_t,N))
v_b = sp.simplify(time_derivative(x_b,N))

# Angular Velocity Vector
w_wl = trans(0, theta_wl.diff(), 0)
w_wr = trans(0, theta_wr.diff(), 0)
w_t = trans(0, theta_p.diff(), 0)
w_c = trans(x_l.diff(), 0, 0)
w_b = trans(0, x.diff()/r, 0)


# Generalized Coordinates
# q = sp.Matrix([[theta_wl], [theta_wr], [x_l], [theta_p]])
q = sp.Matrix([[x], [x_l], [theta_p]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
# T_wl = 0.5*(m_wl*v_wl.dot(v_wl) + I_w*w_wl[1]**2)
# T_wr = 0.5*(m_wr*v_wr.dot(v_wr) + I_w*w_wr[1]**2)
T_w = 0.5*(m_w*v_b.dot(v_b) + I_w*w_b[1]**2)
T_c = 0.5*(m_c*v_c.dot(v_c) + I_c*w_t[1]**2)
T_t = 0.5*(m_t*v_t.dot(v_t) + I_t*w_t[1]**2)

T = T_w + T_c + T_t
# T = T_wl + T_wr + T_c + T_t

# Potential Energy
U_c = m_c*g*(h_c*cos(theta_p))
U_t = m_t*g*(h_t*cos(theta_p)-x_l*sin(theta_p))
U = U_c + U_t

# Lagrangian
Lag = T - U
Lag


                                    2                                         
          2            2   0.5⋅I_w⋅ẋ                                         
0.5⋅I_c⋅θₚ̇  + 0.5⋅Iₜ⋅θₚ̇  + ────────── - g⋅h_c⋅m_c⋅cos(θₚ) - g⋅mₜ⋅(hₜ⋅cos(θₚ)
                                2                                             
                               r                                              

                                                                              
                        ⎛   2    2       2                       2⎞          ⎛
 - xₗ⋅sin(θₚ)) + 0.5⋅m_c⋅⎝h_c ⋅sin (θₚ)⋅θₚ̇  + (h_c⋅cos(θₚ)⋅θₚ̇ + ẋ) ⎠ + 0.5⋅
                                                                              
                                                                              

                                                                              
                                             2                                
mₜ⋅⎝(-hₜ⋅sin(θₚ)⋅θₚ̇ - xₗ⋅cos(θₚ)⋅θₚ̇ - sin(θₚ)⋅xₗ

In [4]:
x_b, x_c, x_t

⎛⎡x⎤  ⎡h_c⋅sin(θₚ) + x⎤  ⎡hₜ⋅sin(θₚ) + x + xₗ⋅cos(θₚ)⎤⎞
⎜⎢ ⎥  ⎢               ⎥  ⎢                           ⎥⎟
⎜⎢0⎥, ⎢       0       ⎥, ⎢             0             ⎥⎟
⎜⎢ ⎥  ⎢               ⎥  ⎢                           ⎥⎟
⎝⎣0⎦  ⎣  h_c⋅cos(θₚ)  ⎦  ⎣  hₜ⋅cos(θₚ) - xₗ⋅sin(θₚ)  ⎦⎠

In [5]:
tau = get_torque_from_L(Lag,q,qd)
tau = sp.simplify(tau)
tau
# Ml, Cl, Gl, Wl = ir.get_EoM_from_T(tau,qdd,g)
# Ml, Cl, Gl, Wl

⎡                                    2 ⎛    ⎛                2                
⎢                       1.0⋅I_w⋅ẍ + r ⋅⎝m_c⋅⎝- h_c⋅sin(θₚ)⋅θₚ̇  + h_c⋅cos(θₚ)
⎢                       ──────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣1.0⋅I_c⋅θₚ̈ + 1.0⋅Iₜ⋅θₚ̈ - g⋅h_c⋅m_c⋅sin(θₚ) - g⋅hₜ⋅mₜ⋅sin(θₚ) - g⋅mₜ⋅xₗ⋅cos(

     ⎞      ⎛             2                                                2  
⋅θₚ̈ + ẍ⎠ - mₜ⋅⎝hₜ⋅sin(θₚ)⋅θₚ̇  - hₜ⋅cos(θₚ)⋅θₚ̈ +

In [6]:
# F= sp.symbols('F')
# u = sp.Matrix([F])

# u_matrix = sp.Matrix([[0],[0],[-F]])

F, T, T_wl, T_wr= sp.symbols('F, T ,T_wl, T_wr')
# u = sp.Matrix([[T_wl],[T_wr],[F]])
u = sp.Matrix([[T],[F]])

# u_matrix = sp.Matrix([[-T_wl], [-T_wr], [T_wl+T_wr], [-F]])
u_matrix = sp.Matrix([[-T], [-F], [T]])
# u_matrix = sp.Matrix([[-T_wl], [-T_wr], [-F], [0]])

tau_eq = tau+u_matrix
tau_eq


⎡                                        2 ⎛    ⎛                2            
⎢                           1.0⋅I_w⋅ẍ + r ⋅⎝m_c⋅⎝- h_c⋅sin(θₚ)⋅θₚ̇  + h_c⋅cos
⎢                      -T + ──────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣1.0⋅I_c⋅θₚ̈ + 1.0⋅Iₜ⋅θₚ̈ + T - g⋅h_c⋅m_c⋅sin(θₚ) - g⋅hₜ⋅mₜ⋅sin(θₚ) - g⋅mₜ⋅xₗ⋅

         ⎞      ⎛             2                                               
(θₚ)⋅θₚ̈ + ẍ⎠ - mₜ⋅⎝hₜ⋅sin(θₚ)⋅θₚ̇  - hₜ⋅cos(θₚ)⋅θ

In [7]:
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u)

In [8]:
M, C, G, W

⎛⎡           1.0⋅I_w                                                          
⎜⎢           ─────── + m_c + 1.0⋅mₜ + 1.0⋅m_w              1.0⋅mₜ⋅cos(θₚ)     
⎜⎢               2                                                            
⎜⎢              r                                                             
⎜⎢                                                                            
⎜⎢                    1.0⋅mₜ⋅cos(θₚ)                           1.0⋅mₜ         
⎜⎢                                                                            
⎜⎢                                                                            
⎝⎣1.0⋅h_c⋅m_c⋅cos(θₚ) + 1.0⋅hₜ⋅mₜ⋅cos(θₚ) - mₜ⋅xₗ⋅sin(θₚ)    1.0⋅hₜ⋅mₜ     1.0

                                                      ⎤                       
h_c⋅m_c⋅cos(θₚ) + 1.0⋅mₜ⋅(hₜ⋅cos(θₚ) - xₗ⋅sin(θₚ))    ⎥  ⎡-(1.0⋅h_c⋅m_c⋅sin(θₚ
                                                      ⎥  ⎢                    
                                                   

In [9]:
# linearlize_eq = {sp.sin(2*theta_p):2*theta_p, sp.sin(theta_p):theta_p, sp.cos(2*theta_p):1, sp.cos(theta_p):1, theta_p.diff():0, x_l.diff():0, x_l:0}
linearlize_eq = {sp.sin(theta_p):theta_p, sp.cos(theta_p):1, theta_p.diff():0, x_l:0}
Ml =M.subs(linearlize_eq)
Cl =C.subs(linearlize_eq)
Gl =G.subs(linearlize_eq)
Wl =W.subs(linearlize_eq)
Ml, Cl, Gl, Wl

⎛⎡1.0⋅I_w                                                                     
⎜⎢─────── + m_c + 1.0⋅mₜ + 1.0⋅m_w   1.0⋅mₜ                h_c⋅m_c + 1.0⋅hₜ⋅mₜ
⎜⎢    2                                                                       
⎜⎢   r                                                                        
⎜⎢                                                                            
⎜⎢             1.0⋅mₜ                1.0⋅mₜ                     1.0⋅hₜ⋅mₜ     
⎜⎢                                                                            
⎜⎢                                                                       2    
⎝⎣    1.0⋅h_c⋅m_c + 1.0⋅hₜ⋅mₜ       1.0⋅hₜ⋅mₜ  1.0⋅I_c + 1.0⋅Iₜ + 1.0⋅h_c ⋅m_c

             ⎤                                                ⎞
             ⎥                                                ⎟
             ⎥  ⎡0⎤  ⎡              0               ⎤  ⎡1   0⎤⎟
             ⎥  ⎢ ⎥  ⎢                              ⎥  ⎢     ⎥⎟
             ⎥, ⎢0⎥, ⎢         -

In [10]:
# param = {m_t: 8.885, m_c:4.124, m_wl:2, m_wr:2, r:0.065, h_c:0.034617, h_t:0.11198, I_t:0.214776135, I_c:0.003414834, I_w:0.004647634, g:9.81}
# param = {m_t: 368.2986, m_c:31.914, m_wl:9.595, m_wr:9.595, r:0.2206625, h_c:0.05774, h_t:0.28433, I_t:179.5035, I_c:0.4868, I_w:0.1367, g:9.81}

# param = {r:0.065, h_c:0.034617, h_t:0.11198, m_w:2, m_wl:2, m_wr:2, m_c:4.124, m_t:8.885, g:9.8}
# param = {m_t: 368.2986, m_c:31.914, m_w:9.595, r:0.2206625, h_c:0.05774, h_t:0.28433, I_t:179.5035, I_c:0.4868, I_w:0.1367, g:9.81}
param = {m_t: 307.332, m_c:38.72018, m_w:12.05514,
         r:0.22, h_c:0.00203, h_t:0.30504,
         I_t:175.3832154, I_c:1.154265591, I_w:0.189170789, g:9.81}

# Moment of Inertia
param[I_w] = 0.189170789 #0.1367 #0.2734 #0.1367
param[I_c] = 1.154265591 #0.4868 #0.6541
param[I_t] = 175.3832154 #179.5035 #173.4619

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

# Forward Dynamics eqation
# qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
Mlp_inv = simplify(Mlp.inv())
qdd_rhs_A = simplify(Mlp_inv*(-Clp -Glp))
qdd_rhs_B = simplify(Mlp_inv*Wlp*u)

# Nonlinear Dynamics Model
Mp = msubs(M, param)
Cp = msubs(C, param)
Gp = msubs(G, param)
Wp = msubs(W, param)
# Forward Dynamics eqation
# qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
Mp_inv = (Mp.inv())
qdd_rhs_A_nonL = (Mp_inv*(-Cp -Gp))
qdd_rhs_B_nonL = (Mp_inv*Wp*u)
qdd_rhs_A_nonL, qdd_rhs_B_nonL

# State-space equation. 
X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

# State-space equation
A = simplify(Xd_A.jacobian(X))
B = simplify(Xd_B.jacobian(U))
C = simplify(X.jacobian(X))
D = simplify(X.jacobian(U))

ss0 = [A, B, C, D]
sys0 = control.ss(*[pl.array(mat_i.subs(param)).astype(float) for mat_i in ss0])
mprint(sys0)

<LinearIOSystem>: sys[2]
Inputs (2): ['u[0]', 'u[1]']
Outputs (6): ['y[0]', 'y[1]', 'y[2]', 'y[3]', 'y[4]', 'y[5]']
States (6): ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]']

A = [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
       0.00000000e+00  0.00000000e+00]
     [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
       1.00000000e+00  0.00000000e+00]
     [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
       0.00000000e+00  1.00000000e+00]
     [ 0.00000000e+00  0.00000000e+00 -5.51338566e+01  0.00000000e+00
       0.00000000e+00  0.00000000e+00]
     [ 0.00000000e+00  0.00000000e+00  6.49350361e+01  0.00000000e+00
       0.00000000e+00  0.00000000e+00]
     [ 0.00000000e+00  0.00000000e+00  2.89157301e-02  0.00000000e+00
       0.00000000e+00  0.00000000e+00]]

B = [[ 0.          0.        ]
     [ 0.          0.        ]
     [ 0.          0.        ]
     [ 0.0182951  -0.01828448]
     [-0.01656471  0.02206288]
     [-0.00

In [11]:
sys0.B

matrix([[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.0182951 , -0.01828448],
        [-0.01656471,  0.02206288],
        [-0.00567266, -0.00171976]])

In [12]:
inv_dyn = get_Simplified_EoM(tau_eq, q)
inv_dyn

⎡     ⎛                    2                2           2                2    
⎢-2.0⋅⎝-0.5⋅I_w⋅ẍ + 0.5⋅T⋅r  + 0.5⋅h_c⋅m_c⋅r ⋅sin(θₚ)⋅θₚ̇  - 0.5⋅h_c⋅m_c⋅r ⋅c
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                    1.0⋅hₜ⋅mₜ⋅xₗ̈ + 2.0⋅mₜ⋅xₗ⋅θₚ̇⋅xₗ̇ + 1.0⋅(

                     2           2              2                       2     
os(θₚ)⋅θₚ̈ + 0.5⋅hₜ⋅mₜ⋅r ⋅sin(θₚ)⋅θₚ̇  - 0.5⋅hₜ⋅mₜ⋅

In [13]:
# Mass :0kg, theta : 0deg, x_dot : 0m/s 일때 평형 방정식
ss_eq1 = {theta_p:0, theta_p.diff():0, theta_p.diff().diff():0, x.diff():0, x.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
lin_ss_eq1 = msubs(inv_dyn, ss_eq1)
lin_ss_eq1


⎡    -1.0⋅T     ⎤
⎢               ⎥
⎢    -1.0⋅F     ⎥
⎢               ⎥
⎣1.0⋅T - g⋅mₜ⋅xₗ⎦

In [14]:
# Mass :0kg
ss_eq2 = {theta_p.diff():0, theta_p.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
lin_ss_eq2 = msubs(inv_dyn, ss_eq2)
lin_ss_eq2


⎡                            ⎛                    2            2             2
⎢                       -2.0⋅⎝-0.5⋅I_w⋅ẍ + 0.5⋅T⋅r  - 0.5⋅m_c⋅r ⋅ẍ - 0.5⋅mₜ⋅
⎢                       ──────────────────────────────────────────────────────
⎢                                                            2                
⎢                                                           r                 
⎢                                                                             
⎢                                         -F - g⋅mₜ⋅sin(θₚ) + 1.0⋅mₜ⋅cos(θₚ)⋅x
⎢                                                                             
⎣1.0⋅T - g⋅h_c⋅m_c⋅sin(θₚ) - g⋅hₜ⋅mₜ⋅sin(θₚ) - g⋅mₜ⋅xₗ⋅cos(θₚ) + 1.0⋅(h_c⋅m_c⋅

              2  ⎞                        ⎤
r ⋅ẍ - 0.5⋅m_w⋅r ⋅ẍ⎠                        ⎥
───────────────────                       ⎥
                                          ⎥
                                          ⎥
                                          ⎥
̈                   

In [15]:
solve(lin_ss_eq2[2], x_l)

⎡T - g⋅h_c⋅m_c⋅sin(θₚ) - g⋅hₜ⋅mₜ⋅sin(θₚ) + h_c⋅m_c⋅cos(θₚ)⋅ẍ + hₜ⋅mₜ⋅cos(θₚ)⋅
⎢─────────────────────────────────────────────────────────────────────────────
⎣                         mₜ⋅(g⋅cos(θₚ) + sin(θₚ)⋅ẍ)                         

ẍ⎤
⎥
⎦

In [16]:
solve(lin_ss_eq2[0], T), solve(lin_ss_eq2[1], F)

⎛⎡⎛       2                 ⎞  ⎤                               ⎞
⎜⎢⎝I_w + r ⋅(m_c + mₜ + m_w)⎠⋅ẍ⎥                               ⎟
⎜⎢─────────────────────────────⎥, [mₜ⋅(-g⋅sin(θₚ) + cos(θₚ)⋅ẍ)]⎟
⎜⎢               2             ⎥                               ⎟
⎝⎣              r              ⎦                               ⎠

In [17]:
# SIP Model

In [18]:
# Displacement Vector
# x_wl = trans(r*theta_wl,0,0)
# x_wr = trans(r*theta_wr,0,0)
# x_c = (x_wr + x_wl)/2 + rot_y(theta_p) @ trans(0,0,h_c)
# x_t = (x_wr + x_wl)/2 + rot_y(theta_p) @ trans(x_l,0,h_t)
x_b = trans(x,0,0)
x_c = x_b + rot_y(theta_p) @ trans(0,0,h_c)

# Velocity Vector
# v_wl = sp.simplify(time_derivative(x_wl,N))
# v_wr = sp.simplify(time_derivative(x_wr,N))
v_b = sp.simplify(time_derivative(x_b,N))
v_c = sp.simplify(time_derivative(x_c,N))

# Angular Velocity Vector
w_t = trans(0, theta_p.diff(), 0)
w_b = trans(0, x.diff()/r, 0)


# Generalized Coordinates
# q = sp.Matrix([[theta_wl], [theta_wr], [x_l], [theta_p]])
q = sp.Matrix([[x], [theta_p]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
# T_wl = 0.5*(m_wl*v_wl.dot(v_wl) + I_w*w_wl[1]**2)
# T_wr = 0.5*(m_wr*v_wr.dot(v_wr) + I_w*w_wr[1]**2)
T_w = 0.5*(m_w*v_b.dot(v_b) + I_w*w_b[1]**2)
T_c = 0.5*(m_c*v_c.dot(v_c) + I_c*w_t[1]**2)

T = T_w + T_c
# T = T_wl + T_wr + T_c + T_t

# Potential Energy
U_c = m_c*g*(h_c*cos(theta_p))
U_w = m_w*g*r
U = U_c + U_w

# Lagrangian
Lag = T - U
Lag


                       2                                                      
          2   0.5⋅I_w⋅ẋ                                          ⎛   2    2  
0.5⋅I_c⋅θₚ̇  + ────────── - g⋅h_c⋅m_c⋅cos(θₚ) - g⋅m_w⋅r + 0.5⋅m_c⋅⎝h_c ⋅sin (θ
                   2                                                          
                  r                                                           

                                           
     2                       2⎞            2
ₚ)⋅θₚ̇  + (h_c⋅cos(θₚ)⋅θₚ̇ + ẋ) ⎠ + 0.5⋅m_w⋅ẋ 
                                           
                                           

In [19]:
tau = get_torque_from_L(Lag,q,qd)
tau = sp.simplify(tau)
tau

⎡             2 ⎛    ⎛                2                     ⎞            ⎞⎤
⎢1.0⋅I_w⋅ẍ + r ⋅⎝m_c⋅⎝- h_c⋅sin(θₚ)⋅θₚ̇  + h_c⋅cos(θₚ)⋅θₚ̈ + ẍ⎠ + 1.0⋅m_w⋅ẍ⎠⎥
⎢─────────────────────────────────────────────────────────────────────────⎥
⎢                                     2                                   ⎥
⎢                                    r                                    ⎥
⎢                                                                         ⎥
⎢                                        2                                ⎥
⎣1.0⋅I_c⋅θₚ̈ - g⋅h_c⋅m_c⋅sin(θₚ) + 1.0⋅h_c ⋅m_c⋅θₚ̈ + 1.0⋅h_c⋅m_c⋅cos(θₚ)⋅ẍ ⎦

In [20]:
# F= sp.symbols('F')
# u = sp.Matrix([F])

# u_matrix = sp.Matrix([[0],[0],[-F]])

F, T = sp.symbols('F, T')

u_matrix = sp.Matrix([[-T], [T]])

tau_eq = tau+u_matrix
tau_eq


⎡                  2 ⎛    ⎛                2                     ⎞            
⎢     1.0⋅I_w⋅ẍ + r ⋅⎝m_c⋅⎝- h_c⋅sin(θₚ)⋅θₚ̇  + h_c⋅cos(θₚ)⋅θₚ̈ + ẍ⎠ + 1.0⋅m
⎢-T + ────────────────────────────────────────────────────────────────────────
⎢                                          2                                  
⎢                                         r                                   
⎢                                                                             
⎢                                             2                               
⎣ 1.0⋅I_c⋅θₚ̈ + T - g⋅h_c⋅m_c⋅sin(θₚ) + 1.0⋅h_c ⋅m_c⋅θₚ̈ + 1.0⋅h_c⋅m_c⋅cos(θₚ)

⎞⎤
_w⋅ẍ⎠⎥
─⎥
 ⎥
 ⎥
 ⎥
 ⎥
⋅ẍ ⎦

In [21]:
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u)

In [22]:
M, C, G, W

⎛⎡1.0⋅I_w                                        ⎤                            
⎜⎢─────── + m_c + 1.0⋅m_w     h_c⋅m_c⋅cos(θₚ)    ⎥  ⎡                   2⎤    
⎜⎢    2                                          ⎥  ⎢-h_c⋅m_c⋅sin(θₚ)⋅θₚ̇ ⎥  ⎡
⎜⎢   r                                           ⎥, ⎢                    ⎥, ⎢ 
⎜⎢                                               ⎥  ⎣         0          ⎦  ⎣-
⎜⎢                                          2    ⎥                            
⎝⎣  1.0⋅h_c⋅m_c⋅cos(θₚ)    1.0⋅I_c + 1.0⋅h_c ⋅m_c⎦                            

                               ⎞
                               ⎟
          0           ⎤  ⎡1   0⎤⎟
                     ⎥, ⎢     ⎥⎟
1.0⋅g⋅h_c⋅m_c⋅sin(θₚ)⎦  ⎣-1  0⎦⎟
                               ⎟
                               ⎠

In [23]:
# linearlize_eq = {sp.sin(2*theta_p):2*theta_p, sp.sin(theta_p):theta_p, sp.cos(2*theta_p):1, sp.cos(theta_p):1, theta_p.diff():0, x_l.diff():0, x_l:0}
linearlize_eq = {sp.sin(theta_p):theta_p, sp.cos(theta_p):1, theta_p.diff():0}
Ml =M.subs(linearlize_eq)
Cl =C.subs(linearlize_eq)
Gl =G.subs(linearlize_eq)
Wl =W.subs(linearlize_eq)
Ml, Cl, Gl, Wl

⎛⎡1.0⋅I_w                                        ⎤                            
⎜⎢─────── + m_c + 1.0⋅m_w         h_c⋅m_c        ⎥                            
⎜⎢    2                                          ⎥  ⎡0⎤  ⎡        0        ⎤  
⎜⎢   r                                           ⎥, ⎢ ⎥, ⎢                 ⎥, 
⎜⎢                                               ⎥  ⎣0⎦  ⎣-1.0⋅g⋅h_c⋅m_c⋅θₚ⎦  
⎜⎢                                          2    ⎥                            
⎝⎣      1.0⋅h_c⋅m_c        1.0⋅I_c + 1.0⋅h_c ⋅m_c⎦                            

       ⎞
       ⎟
⎡1   0⎤⎟
⎢     ⎥⎟
⎣-1  0⎦⎟
       ⎟
       ⎠

In [24]:
# param = {m_t: 8.885, m_c:4.124, m_wl:2, m_wr:2, r:0.065, h_c:0.034617, h_t:0.11198, I_t:0.214776135, I_c:0.003414834, I_w:0.004647634, g:9.81}
# param = {m_t: 368.2986, m_c:31.914, m_wl:9.595, m_wr:9.595, r:0.2206625, h_c:0.05774, h_t:0.28433, I_t:179.5035, I_c:0.4868, I_w:0.1367, g:9.81}

# param = {r:0.065, h_c:0.034617, h_t:0.11198, m_w:2, m_wl:2, m_wr:2, m_c:4.124, m_t:8.885, g:9.8}
# param = {m_t: 368.2986, m_c:31.914, m_w:9.595, r:0.2206625, h_c:0.05774, h_t:0.28433, I_t:179.5035, I_c:0.4868, I_w:0.1367, g:9.81}
param = {m_c: 346.052, m_w:12.05514,
         r:0.22, h_c:0.2711359, g:9.81}

# Moment of Inertia
param[I_w] = 0.189170789 #0.1367 #0.2734 #0.1367
param[I_c] = 179.6951 #0.4868 #0.6541

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

# Forward Dynamics eqation
# qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
Mlp_inv = simplify(Mlp.inv())
qdd_rhs_A = simplify(Mlp_inv*(-Clp -Glp))
qdd_rhs_B = simplify(Mlp_inv*Wlp*u)

# Nonlinear Dynamics Model
Mp = msubs(M, param)
Cp = msubs(C, param)
Gp = msubs(G, param)
Wp = msubs(W, param)
# Forward Dynamics eqation
# qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
Mp_inv = (Mp.inv())
qdd_rhs_A_nonL = (Mp_inv*(-Cp -Gp))
qdd_rhs_B_nonL = (Mp_inv*Wp*u)
qdd_rhs_A_nonL, qdd_rhs_B_nonL

# State-space equation. 
X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

# State-space equation
A = simplify(Xd_A.jacobian(X))
B = simplify(Xd_B.jacobian(U))
C = simplify(X.jacobian(X))
D = simplify(X.jacobian(U))

ss0 = [A, B, C, D]
sys0 = control.ss(*[pl.array(mat_i.subs(param)).astype(float) for mat_i in ss0])
mprint(sys0)

<LinearIOSystem>: sys[3]
Inputs (2): ['u[0]', 'u[1]']
Outputs (4): ['y[0]', 'y[1]', 'y[2]', 'y[3]']
States (4): ['x[0]', 'x[1]', 'x[2]', 'x[3]']

A = [[ 0.          0.          1.          0.        ]
     [ 0.          0.          0.          1.        ]
     [ 0.         -1.31934817  0.          0.        ]
     [ 0.          5.09047548  0.          0.        ]]

B = [[ 0.          0.        ]
     [ 0.          0.        ]
     [ 0.0045672   0.        ]
     [-0.00696384  0.        ]]

C = [[1. 0. 0. 0.]
     [0. 1. 0. 0.]
     [0. 0. 1. 0.]
     [0. 0. 0. 1.]]

D = [[0. 0.]
     [0. 0.]
     [0. 0.]
     [0. 0.]]

